In [1]:
import glob2
import os
import bs4
import nltk
from transformers import BertTokenizer

/anaconda3/envs/DEEPML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/DEEPML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/DEEPML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/DEEPML/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [10]:
os.chdir('/Users/nirmalenduprakash/Documents/Project/NLP/Sentiment Mining')
filenames=glob2.glob('upload/database.mpqa.3.0/gate_anns/**/*.xml')

In [11]:
def getNodeText(text,node1,node2):
    text= text[text.index("\""+node1+"\""):text.index("\""+node2+"\"")]
    nodes_in_text=re.findall("\"\d*\"",text)
    for node in nodes_in_text:
        text=text.replace(node,'')    
    text=text.replace('node','')\
              .replace('>','').replace('<','').replace('id','').replace('=','').replace('\"','')\
             .replace('/','')#.replace(node1,'')#.replace(node2,'')

    return text    

In [12]:
def getSentences(text):
    return nltk.sent_tokenize(text)     

In [13]:
def getNodeSent(sent,txt_with_nodes):
#     print(sent,txt_with_nodes)
    result=''
    search_text=txt_with_nodes
    for word in nltk.word_tokenize(sent):  
        try:
            result=result+str(search_text[:search_text.index(word) + len(word)])      
            search_text=txt_with_nodes[len(result):]
        except:
            continue        
    return result 

In [101]:
documents=[]
for file in filenames:
    with open(file,'r') as f:
        soup=bs4.BeautifulSoup(f)
        txt=soup.select('TextWithNodes')[0].text.replace('\n','').replace('\t','')
        documents.append(txt)

In [62]:
import re
tokenizer = BertTokenizer.from_pretrained(os.curdir)
seq_train=[]
for file in filenames:
    with open(file,'r') as f:
        soup=bs4.BeautifulSoup(f)
        txt=soup.select('TextWithNodes')[0].text
        sents=getSentences(txt)
        txt_with_nodes=str(soup.select('TextWithNodes')[0])    
        agents=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'text':getNodeText(txt_with_nodes,node['startnode'],node['endnode'])}\
                      for node in soup.select("Annotation[Type=agent]")]
        sentiments=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'intensity':str(list(list(node.children)[3])[3]).split('>')[1].split('<')[0],\
                       'type':str(list(list(node.children)[5])[3]).split('>')[1].split('<')[0]}\
                      for node in soup.select("Annotation[Type=attitude]")]
        targets=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'text':getNodeText(txt_with_nodes,node['startnode'],node['endnode'])}\
                      for node in soup.select("Annotation[Type=sTarget]")]
        for indx,sent in enumerate(sents):
            nodeText=getNodeSent(sent,txt_with_nodes)        
            nodes_in_txt=re.findall('\d+',nodeText)
            sent_agent=None
            sent_target=None
            sent_sentiment_type=None
            sent_sentiment_intensity=None
            
            for node in nodes_in_txt:
                sent_agents=[itm['text'] for itm in agents if itm['startNode']==node]
                if(len(sent_agents)>0):
                    sent_agent=sent_agents[0].replace('  ',' ')
                sent_targets=[itm['text'] for itm in targets if itm['startNode']==node]
                if(len(sent_targets)>0):
                    sent_target=sent_targets[0].replace('  ',' ') 
                sent_intensities=[itm['intensity'] for itm in sentiments if itm['startNode']==node]
                if(len(sent_intensities)>0):
                    sent_sentiment_intensity=sent_intensities[0]
                sent_types=[itm['type'] for itm in sentiments if itm['startNode']==node]
                if(len(sent_types)>0):
                    sent_sentiment_type=sent_types[0]     
#                 print('sent:{}'.format(sent),'agent:{}'.format(sent_agent),'target:{}'.format(sent_target),\
#                       'sentiment type:{}'.format(sent_sentiment_type),\
#                       'sentiment intensity:{}'.format(sent_sentiment_intensity))
            
            txt_with_nodes=txt_with_nodes[len(nodeText):]
            labels=tokenizer.tokenize(sent)
            sent=tokenizer.tokenize(sent)
            
            if(sent_agent is not None):
                n=len(tokenizer.tokenize(sent_agent))
                gms=nltk.ngrams(labels,n)
                for i,g in enumerate(gms):
                    if(' '.join(g).strip()==' '.join(tokenizer.tokenize(sent_agent)).strip()):
                        labels[i:i+len(g)]=['B-AG' if j==0 else 'I-AG' for j in range(len(g))]
                        break
                        
            if(sent_target is not None and (sent_agent is not None and sent_agent.strip()!=sent_target.strip())):
                n=len(tokenizer.tokenize(sent_target))
                gms=nltk.ngrams(labels,n)
                for i,g in enumerate(gms):
                    if(' '.join(g).strip()==' '.join(tokenizer.tokenize(sent_target)).strip()):
                        labels[i:i+len(g)]=['B-TG' if j==0 else 'I-TG' for j in range(len(g))]
                        break            
                 
            labels=['O' if w not in ['B-AG','I-AG','B-TG','I-TG'] else w for w in labels]  
            seq_train.append({'sent':sent,'labels':labels,'sentiment_type':sent_sentiment_type,\
                             'sentiment_intensity':sent_sentiment_intensity,\
                              'file_name':file,'agent':sent_agent,'target':sent_target})
#             print(seq_train)
#         break

In [63]:
import pandas as pd
df=pd.DataFrame(seq_train)
df.head()

,sent,labels,sentiment_type,sentiment_intensity,file_name,agent,target
0,"[following, the, end, of, the, cold, war, ,, t...","[O, O, O, O, O, O, O, O, B-AG, I-AG, I-AG, O, ...",arguing-pos,medium,upload/database.mpqa.3.0/gate_anns/20011208/20...,the United States,the trend of pragmatism in its foreign policy
1,"[pr, ##ag, ##mat, ##ism, is, ,, in, the, first...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",arguing-pos,low-medium,upload/database.mpqa.3.0/gate_anns/20011208/20...,None,Pragmatism
2,"[after, coming, to, power, ,, un, ##ila, ##ter...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",somewhat-uncertain,a100,upload/database.mpqa.3.0/gate_anns/20011208/20...,Bush,Bush
3,"[alternate, use, of, and, complementary, natur...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",arguing-pos,low-medium,upload/database.mpqa.3.0/gate_anns/20011208/20...,None,Alternate use of and complementary nature of p...
4,"[once, coming, to, power, ,, bush, insisted, o...","[O, O, O, O, O, B-AG, O, O, B-TG, I-TG, I-TG, ...",sentiment-pos,low-medium,upload/database.mpqa.3.0/gate_anns/20011208/20...,Bush,deploying the National Missile Defense System


In [71]:
import numpy as np
max_len_lbl=np.max([df['labels'].apply(lambda x: len(x))])
max_len_sent=np.max([df['sent'].apply(lambda x: len(x))])
# df['sent']=df['sent'].apply(lambda x:['[CLS]'+x+'[SEP]'])
# df['labels']=df['labels'].apply(lambda x:['[CLS]'+x+'[SEP]'])
max_len=max(max_len_lbl,max_len_sent)

In [72]:
def pad(tokens,max_len,cls=False):
  if(len(tokens)<max_len):
    if(cls):
        return ['[CLS]']+tokens+['[PAD]' for _ in range(max_len-len(tokens)-1)]
    else:
        return tokens+['[PAD]' for _ in range(max_len-len(tokens))]
  else:
    return tokens[:max_len] 

In [73]:
df['labels']=df['labels'].apply(lambda x: pad([l for l in x],max_len))
df['sent']=df['sent'].apply(lambda x: pad([l for l in x],max_len,True))

In [74]:
df['encoding']=df['sent'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [75]:
df['attn_mask']=df['encoding'].apply(lambda x: [0 if tok==str('[PAD]') else 1 for tok in x ])

In [76]:
df['sentiment_intensity']=df['sentiment_intensity'].apply(lambda x: x if x in ['medium', 'low-medium',\
                                                        'medium-high', 'low', 'high-extreme', 'high'] else None)

In [77]:
df['sentiment_type']=df['sentiment_type'].apply(lambda x: x if x in ['arguing-pos', 'somewhat-uncertain', 
       'sentiment-pos','intention-pos', 'sentiment-neg', 'other-attitude',
       'very-uncertain', 'arguing-neg', 'intention-neg', 'agree-pos','agree-neg'] else None)

In [78]:
print(len(df))
print(df.isna().sum())

1705
sent                     0
labels                   0
sentiment_type         763
sentiment_intensity    852
file_name                0
agent                  812
target                 779
encoding                 0
attn_mask                0
dtype: int64


In [81]:
# df['agent'].fillna('author',inplace=True)
df.dropna(inplace=True)
print(len(df))

783


In [102]:
import pickle
with open('/Users/nirmalenduprakash/Documents/Project/NLP/Sentiment Mining/documents.pkl','wb') as f:
    pickle.dump(documents,f)
with open('/Users/nirmalenduprakash/Documents/Project/NLP/Sentiment Mining/data.pkl','wb') as f:
    pickle.dump(df,f)

In [105]:
with open('/Users/nirmalenduprakash/Documents/Project/NLP/Sentiment Mining/sample_document.txt','w') as f:
    f.write(documents[0])